# Amazon SageMaker XGBoost Bring Your Own Model
_**Hosting a Pre-Trained scikit-learn Model in Amazon SageMaker XGBoost Algorithm Container**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Optionally, train a scikit learn XGBoost model](#Optionally,-train-a-scikit-learn-XGBoost-model)
1. [Upload the pre-trained model to S3](#Upload-the-pre-trained-model-to-S3)
1. [Set up hosting for the model](#Set-up-hosting-for-the-model)
1. [Validate the model for use](#Validate-the-model-for-use)




---
## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-existing scikit-learn trained XGBoost model with the Amazon SageMaker XGBoost Algorithm container to quickly create a hosted endpoint for that model. Please note that scikit-learn XGBoost model is compatible with SageMaker XGBoost container, whereas other gradient boosted tree models (such as one trained in SparkML) are not.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data. See the documentation for how to specify these.
* The S3 bucket that you want to use for training and model data.

runtime_client = boto3.client('runtime.sagemaker', endpoint_url= 'https://vpce-0803dc74776014d46-iava3t25-us-west-2a.vpce-svc-00a9fd01107f2b64d.us-west-2.vpce.amazonaws.com')

#runtime_client = boto3.client('runtime.sagemaker')


In [41]:
%%time

import os
import boto3
import re
import json
from sagemaker import get_execution_role
import sagemaker
import time

sess = sagemaker.Session()
region = boto3.Session().region_name
role = get_execution_role()
bucket=sess.default_bucket()
#bucket='<s3 bucket>' # put your s3 bucket name here, and create s3 bucket

CPU times: user 147 ms, sys: 17 ms, total: 164 ms
Wall time: 437 ms


In [42]:
prefix = 'sagemaker/DEMO-xgboost-byo'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)
# customize to your bucket where you have stored the data

## Optionally, train a scikit learn XGBoost model

These steps are optional and are needed to generate the scikit-learn model that will eventually be hosted using the SageMaker Algorithm contained. 

### Install XGboost
Note that for conda based installation, you'll need to change the Notebook kernel to the environment with conda and Python3. 

In [43]:
!pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Fetch the dataset

In [44]:
%%time
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

CPU times: user 773 ms, sys: 288 ms, total: 1.06 s
Wall time: 2.81 s


### Prepare the dataset for training

In [45]:
%%time

import struct
import io
import boto3

def get_dataset():
  import pickle
  import gzip
  with gzip.open('mnist.pkl.gz', 'rb') as f:
      u = pickle._Unpickler(f)
      u.encoding = 'latin1'
      return u.load()

CPU times: user 19 µs, sys: 0 ns, total: 19 µs
Wall time: 23.1 µs


In [46]:
train_set, valid_set, test_set = get_dataset()

train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]

### Train the XGBClassifier

In [47]:
import xgboost as xgb
import sklearn as sk 

bt = xgb.XGBClassifier(max_depth=5,
                       learning_rate=0.2,
                       n_estimators=10,
                       objective='multi:softmax')   # Setup xgboost model
#900597767885.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:beta_low_lat_inf



bt.fit(train_X, train_y, # Train it to our data
       eval_set=[(valid_X, valid_y)], 
       verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
bt.get_xgb_params()

{'objective': 'multi:softprob',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'interaction_constraints': '',
 'learning_rate': 0.2,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'monotone_constraints': '()',
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': None,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

### Save the trained model file
Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file also need to tar-zipped. 

In [49]:
model_file_name = "xgboost-model-"
bt._Booster.save_model(model_file_name)

In [50]:
!tar czvf model.tar.gz $model_file_name

xgboost-model-


## Upload the pre-trained model to S3

In [51]:
fObj = open("model.tar.gz", 'rb')
key= os.path.join(prefix, model_file_name, 'model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

## Set up hosting for the model in your VPC/Subnet with Security Groups

### Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.

In [52]:
from sagemaker.amazon.amazon_estimator import get_image_uri
#container = get_image_uri(boto3.Session().region_name, 'xgboost')
#container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')
#container = '900597767885.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:beta_low_lat_inf'
container = '878107166805.dkr.ecr.us-west-2.amazonaws.com/preprod-xgboost-framework:fruitstand_bjoern'
print('container image: {}'.format(container))

container image: 878107166805.dkr.ecr.us-west-2.amazonaws.com/preprod-xgboost-framework:fruitstand_bjoern


In [53]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)
sm_client = boto3.client('sagemaker')

print ('model_url: {}'.format(model_url))

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
     
}

vpc = 'vpc-027eb36a3c830a37e'
subnet_a = 'subnet-0c02841a81fce0d28'
subnet_b = 'subnet-0d855fab759d3efbb'
security_group = 'sg-0b0fe4c5610c5ba2a'

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container
#    ,
#    VpcConfig={
#        'SecurityGroupIds': [security_group],
#        'Subnets': [subnet_a, subnet_b],
#    },
#    EnableNetworkIsolation=True
    )

print(create_model_response['ModelArn'])
print(model_name)

model_url: https://s3-us-west-2.amazonaws.com/sagemaker-us-west-2-328296961357/sagemaker/DEMO-xgboost-byo/xgboost-model-/model.tar.gz
arn:aws:sagemaker:us-west-2:328296961357:model/xgboost-model-2020-05-22-21-27-29
xgboost-model-2020-05-22-21-27-29
CPU times: user 22.9 ms, sys: 0 ns, total: 22.9 ms
Wall time: 234 ms


### Create endpoint configuration for a single model

SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, you can create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [54]:
from time import gmtime, strftime

endpoint_config_name_prod = 'XGBoostEP-Prod-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name_prod)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name_prod,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge', 
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'model-Blue'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

XGBoostEP-Prod-2020-05-22-21-27-29
Endpoint Config Arn: arn:aws:sagemaker:us-west-2:328296961357:endpoint-config/xgboostep-prod-2020-05-22-21-27-29


### Create endpoint
You create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 8-10 minutes to complete. 

#### First, let us create a helper funciton to poll for endoint status

In [55]:
def poll_endpoint(endpoint_name, endpoint_ARN, poll_period=60):
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

    while (status=='Creating' or status=='Updating'):
        time.sleep(poll_period)
        resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = resp['EndpointStatus']
        print("Status: " + status)

    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)  

#### Now, let us create the endpoint and poll until it's status changes to "InService"

In [56]:
%%time
import time

endpoint_name = 'XGBoostEP-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name_prod)
endpoint_ARN = create_endpoint_response['EndpointArn']
print(endpoint_ARN)

poll_endpoint(endpoint_name=endpoint_name, endpoint_ARN=endpoint_ARN)

XGBoostEP-2020-05-22-21-27-29
arn:aws:sagemaker:us-west-2:328296961357:endpoint/xgboostep-2020-05-22-21-27-29
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:328296961357:endpoint/xgboostep-2020-05-22-21-27-29
Status: InService
CPU times: user 96.1 ms, sys: 4.04 ms, total: 100 ms
Wall time: 7min 1s


## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint. You can take the below code and use it as a basis for endpoint invocation from a utility EC2 machine or a different SageMaker notebook.

In [57]:
runtime_client = boto3.client('runtime.sagemaker')

Lets generate the prediction for a single datapoint. We'll pick one from the test data generated earlier.

In [58]:
import numpy as np
point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")

In [61]:
%%time
import json

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload,
                                   CustomAttributes = "arbitrary_customaizable_string")
print (response)
result = response['Body'].read().decode('ascii')
print('Predicted Class Probabilities: {}.'.format(result))

{'ResponseMetadata': {'RequestId': 'c4eea5d9-119d-47ee-9910-4a4297b808d1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c4eea5d9-119d-47ee-9910-4a4297b808d1', 'x-amzn-invoked-production-variant': 'model-Blue', 'date': 'Fri, 22 May 2020 21:51:14 GMT', 'content-type': 'text/csv', 'content-length': '2'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv', 'InvokedProductionVariant': 'model-Blue', 'Body': <botocore.response.StreamingBody object at 0x7fc5fa2040f0>}
Predicted Class Probabilities: [].
CPU times: user 9.36 ms, sys: 3.94 ms, total: 13.3 ms
Wall time: 52.4 ms


In [90]:
import logging
import threading
import time, timeit

def thread_function(name, N=100):
    response = """\
runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
"""
    #SageMaker endpoint warm-up
    for i in range(5):
        timeit.timeit(stmt=response, globals=globals(), number=1)
        time.sleep(0.1)

    #end-2-end client latency measurement
    timeit_latency_1=timeit.timeit(stmt=response, globals=globals(), number=1)
    timeit_latency_N=timeit.timeit(stmt=response, globals=globals(), number=N)
    print("single timeit() invocation latency {:.4f} sec".format(timeit_latency_1))
    print("average timeit() latency for {} invocations: {:.4f} sec".format(N, timeit_latency_N/N)) 
    

In [91]:
invoke_1 = threading.Thread(target=thread_function, args=(100,))

In [92]:
invoke_1.start()

In [93]:
print("temp_print")

temp_print


In [94]:
invoke_1.join()

single timeit() invocation latency 0.0180 sec
average timeit() latency for 100 invocations: 0.0201 sec


In [95]:
print("the end")

the end


### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [96]:
#sm_client.delete_endpoint(EndpointName=endpoint_name)